<a href="https://colab.research.google.com/github/massone99/visione_artificiale_colab_notebooks/blob/main/SemanticSegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Esercitazione su semantic segmentation**
Nell'esercitazione odierna utilizzeremo l'architettura U-Net per segmentare i nuclei delle cellule in immagini medicali.

Faremo uso del framework **TensorFlow**, sfruttando la libreria open-source **Keras** appositamente progettata per permettere una rapida prototipazione di reti neurali profonde.

# **Operazioni preliminari**

Eseguendo la cella sottostante tutto il materiale necessario per lo svolgimento dell'esercitazione verrà scaricato sulla macchina remota. Alla fine dell'esecuzione selezionare il tab **Files** per verificare che tutto sia stato scaricato correttamente.

In [ ]:
!wget http://bias.csr.unibo.it/VR/Esercitazioni/DBs/SemanticSegmentation/Train_2018DataScienceBowl.zip
!wget http://bias.csr.unibo.it/VR/Esercitazioni/DBs/SemanticSegmentation/Test_2018DataScienceBowl.zip
!wget http://bias.csr.unibo.it/VR/Esercitazioni/PythonUtilities.zip

!unzip /content/Test_2018DataScienceBowl.zip
!unzip /content/Train_2018DataScienceBowl.zip
!unzip /content/PythonUtilities.zip

!rm /content/Test_2018DataScienceBowl.zip
!rm /content/Train_2018DataScienceBowl.zip
!rm /content/PythonUtilities.zip

# **Import delle librerie di base**
Per prima cosa è necessario eseguire l'import delle librerie di base utilizzate durante l'esecitazione.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import sklearn
from sklearn.model_selection import train_test_split
import random

import vr_utilities

# **Dataset per l'addestramento**
In questa esercitazione useremo il dataset di training fornito per la  competizione [*2018 Data Science Bowl*](https://www.kaggle.com/c/data-science-bowl-2018) presente su [**Kaggle**](https://www.kaggle.com/), una piattaforma online per competizioni di modelli predittivi e analitici.

Il dataset consiste in un insieme di immagini RGB e rispettive maschere (*ground truth*) per la segmentazione di nuclei cellulari (problema biclasse: 0=background, 1=nucleo).

Eseguendo la cella sottostante il dataset di training sarà caricato in memoria.

In [ ]:
db_path='/content/Train'
image_ext='.bmp'

# Caricamento delle immagini
print('Caricamento in corso ...')
images,masks=vr_utilities.load_image_dataset_with_masks(db_path+'/Images',db_path+'/Masks','*'+image_ext)

# Normalizzazione
images=(images/255)
masks=(masks/255)
images=images.astype(np.float32)
masks=masks.astype(np.float32)

print('Shape immagini dataset: ',images.shape)
print('Shape maschere dataset: ',masks.shape)

La cella seguente contiene il codice per mostrare un'immagine del training set scelta casualmente e rispettiva maschera (nero=background e bianco=nucleo).

In [ ]:
# Visualizzazione di un'immagine casuale
random_idx=random.randint(0,images.shape[0])
_, axs = plt.subplots(1, 2,figsize=(10, 5))
axs[0].imshow(images[random_idx]),axs[0].axis('off'),axs[0].set_title('Image')
axs[1].imshow(np.squeeze(masks[random_idx],axis=2),cmap='gray'),axs[1].axis('off'),axs[1].set_title('Mask')
plt.show()

Il dataset caricato può essere suddiviso in due parti: training e validation set. Attraverso il validation set sarà possibile valutare i risultati della rete addestrata sul training set al fine di individuare il valore ottimale per gli iperparametri.

Visto che in Machine Learning è comune eseguire tale operazione, la libreria **Scikit-learn** mette a disposizione una apposita funzione, [**train_test_split(...)**](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), che permette di separare un dataset in due parti.

Il parametro *validation_size* descrive la percentuale (o il numero) di pattern che dovrà essere contenuta nel validation set. Come configurazione predefinita **train_test_split(...)** mescola i pattern al fine di evitare che i dataset restituiti contengano pattern appartenenti solamente a un sottoinsieme delle classi.

In [ ]:
random_seed=1234 #Seme per la selezione pseudo-casuale dei pattern
validation_size=150 #percentuale (o numero) di pattern del validation set

train_x, validation_x, train_y, validation_y = train_test_split(images, masks,
                                                                test_size=validation_size,
                                                                random_state=random_seed)

print('Shape training set:', train_x.shape)
print('Shape validation set:', validation_x.shape)

# **Jaccard index**

Keras, oltre a mettere a disposizione degli utenti [numerose metriche](https://keras.io/metrics/) predefinite per la valutazione delle prestazioni di una rete, permette all'utente di definirne delle proprie nel caso queste non fossero già presenti.

Visto che il problema che dovremo affrontare presenta due classi piuttosto sbilanciate, oltre all'accuratezza sarà utile misurare anche l'indice di *Jaccard* (o *Intersection over Union*):

\begin{align*}
J(A,B)=\frac{|A\cap B|}{|A \cup B|}
\end{align*}

Eseguire la cella sottostante per definire tale indice.

In [ ]:
smooth = 1e-12

def jaccard_index(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)

# **U-Net**

Il modello U-Net è una *fully convolutional neural network* proposta originariamente per effettuare segmentazione binaria (foreground e background) di immagini medicali. È essenzialmente composta da tre parti:
1. **Downsampling Path** - composto da 4 blocchi, ha lo scopo di ridurre le dimensioni spaziali dell'input aumentandone contemporaneamente la profondità. Ogni blocco consiste in due livelli di convoluzione (filtro 3x3, stride=1, nessun padding e ReLU come funzione di attivazione) e un livello di max-pooling (2x2, stride=2 e nessun padding).
2. **Bottleneck** - composto da due livelli di convoluzione (filtro 3x3, stride=1, nessun padding e ReLU come funzione di attivazione).
3. **Upsampling Path** - composto da 4 blocchi, ha lo scopo di portare la *segmentation map* restituita dal *Bottleneck* alle dimensioni spaziali dell'immagine di input. Ogni blocco consiste in un livello di *upsampling* per raddoppiare le dimensioni spaziali, un livello di convoluzione (filtro 2x2, stride=1, nessun padding) che dimezza il numero di canali, la concatenazione (*skip connections*) con la corrispondente feature map del *Downsampling Path* e infine due livelli di convoluzione (filtro 3x3, stride=1, nessun padding e ReLU come funzione di attivazione).

Il livello finale di convoluzione (filtro 1x1, stride=1, nessun padding e *Sigmoid* come funzione di attivazione) è utilizzato per restituire una *segmentation map* con una profondità pari al numero di classi del problema.


![alt text](https://github.com/nikhilroxtomar/UNet-Segmentation-in-Keras-TensorFlow/raw/53b5e697d3c010546b9443534a067541c4c50eb9/images/u-net-architecture.png)

## **Downsampling block**
La cella seguente definisce la funzione **down_block(...)** per creare un blocco di *downsampling* dati in input:
- la feature map (*x*);
- il numero di filtri (*filter_count*).

In [ ]:
def down_block(x, filter_count):
    c = keras.layers.Conv2D(filter_count, 3, padding='same', activation='relu')(x)
    c = keras.layers.Conv2D(filter_count, 3, padding='same', activation='relu')(c)
    p = keras.layers.MaxPool2D(2, 2)(c)
    return c, p

## **Bottleneck**
La cella seguente definisce la funzione **bottleneck(...)** per creare il *bottleneck* dati in input:
- la feature map (*x*);
- il numero di filtri (*filter_count*).

In [ ]:
def bottleneck(x, filter_count):
    c = keras.layers.Conv2D(filter_count, 3, padding='same', activation='relu')(x)
    c = keras.layers.Conv2D(filter_count, 3, padding='same', activation='relu')(c)
    return c

## **Upsampling block**
La cella seguente definisce la funzione **up_block(...)** per creare un blocco di *upsampling* dati in input:
- la feature map (*x*);
- la *skip connection* da concatenare (*skip*);
- il numero di filtri (*filter_count*).

In [ ]:
def up_block(x, skip, filter_count):
    us = keras.layers.UpSampling2D(2)(x)
    c=keras.layers.Conv2D(filter_count, 2, padding='same')(us)
    concat = keras.layers.Concatenate()([c, skip])
    c = keras.layers.Conv2D(filter_count, 3, padding='same', activation='relu')(concat)
    c = keras.layers.Conv2D(filter_count, 3, padding='same', activation='relu')(c)
    return c

## **Definizione del modello**
La cella seguente definisce la funzione **UNet(...)** per creare una rete U-Net (modello) dati in input:
- le dimensioni delle immagini di input (*image_shape*);
- il numero di filtri da utilizzare in ogni blocco di *downsampling* e di *upsampling* (*block_filter_count*).


In [ ]:
def UNet(image_shape,block_filter_count=[64, 128, 256, 512, 1024]):

    inputs = keras.layers.Input(image_shape)

    # Downsampling path
    p0 = inputs
    c1, p1 = down_block(p0, block_filter_count[0])
    c2, p2 = down_block(p1, block_filter_count[1])
    c3, p3 = down_block(p2, block_filter_count[2])
    c4, p4 = down_block(p3, block_filter_count[3])

    # Bottleneck
    bn = bottleneck(p4, block_filter_count[4])

    # Upsampling path
    u1 = up_block(bn, c4, block_filter_count[3])
    u2 = up_block(u1, c3, block_filter_count[2])
    u3 = up_block(u2, c2, block_filter_count[1])
    u4 = up_block(u3, c1, block_filter_count[0])

    # Output
    outputs = keras.layers.Conv2D(1, 1, padding='same', activation='sigmoid')(u4)

    return keras.models.Model(inputs, outputs)

## **Iperparametri**
Gli iperparametri su cui si può intervenire in fase di creazione della rete sono (in ordine di importanza):
1. il numero di filtri da utilizzare in ogni blocco di *downsampling* e di *upsampling* (*block_filter_count*). Nella versione originale della U-Net il numero di filtri per blocco sono: [64, 128, 256, 512, 1024] (da un blocco al successivo il numero di filtri raddoppia). Per riuscire ad addestrare più velocemente la rete si consiglia di partire con un numero inferiore di filtri;
2. l'ottimizzatore (*optimizer*) utilizzato dalla rete per implementare la *back propagation*. Keras mette a disposizione vari [ottimizzatori](https://keras.io/optimizers/) già implementati;
3. i singoli parametri dell'ottimizzatore di cui il più importante è sicuramente il *learning rate*. Per una descrizione dettagliata dei vari parametri di ogni ottimizzatore fare riferimento alla documentazione di Keras;
4. la *loss function* della rete (*loss_function*). Keras mette a disposizione varie [*loss*](https://keras.io/losses/). Attenzione perché non tutte le *loss* sono adatte a un problema binario come quello oggetto dell'esercitazione.

<u>Nota:</u> la cella sottostante assegna agli iperparametri dei valori di partenza utili per fare delle prime prove.

In [ ]:
block_filter_count = [8, 16, 32, 64, 128]

optimizer=keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False)

loss_function='binary_crossentropy'

## **Creazione del modello**
Vediamo ora come creare il modello utilizzando gli iperparametri appena impostati . Richiamando la funzione **UNet(...)** definita in precedenza è possibile istanziare un modello della U-Net mentre, con il metodo [**compile(...)**](https://keras.io/models/model/#compile) della classe [**Model**](https://keras.io/models/model/) si configura il modello per il training dati in input:
- l'optimizer (*optimizer*);
- la *loss* (*loss_function*);
- le metriche che si vogliono monitorare durante il training (nel nostro caso l'accuratezza e l'indice di *Jaccard*).

<u>Nota:</u> prima di eseguire la cella sottostante assicurarsi di aver eseguito la cella precedente altrimenti gli iperparametri non saranno dichiarati.

In [ ]:
# Creazione del modello
model = UNet(train_x[0].shape,block_filter_count)

# Configura il modello per il training
model.compile(optimizer=optimizer, loss=loss_function, metrics=['acc',jaccard_index])

### **Visualizzazione del modello**
Eseguendo la cella seguente è possibile stampare un riepilogo testuale della struttura della rete.

In [ ]:
model.summary()

Se si preferisce una visualizzazione grafica, eseguire la cella seguente.

In [ ]:
tf.keras.utils.plot_model(model,show_shapes=True, show_layer_names=True)

# **Training**
Ora siamo pronti per l'addestramento della U-Net utilizzando le immagini e le maschere del training set (*train_x* e *train_y*) per un numero di epoche pari a *epoch_count* utilizzando *minibatch* di dimensione *batch_size*. Durante l'addestramento le prestazioni della rete verranno valutate anche sul validation set (*validation_x* e *validation_y*).

<u>Nota:</u> *batch_size* dovrebbe essere un divisore del numero di immagini contenute nel training set per evitare che alcune di esse non vengano utilizzate.

In [ ]:
# Numero di epoche di addestramento
epoch_count=5

# Numero di pattern all'interno di ogni minibatch
batch_size=50

Nella cella seguente viene richiamato il metodo [**fit(...)**](https://keras.io/models/model/#fit) che esegue l'intera fase di addestramento in maniera automatica monitorando costantemente la *loss function* e le metriche impostate in fase di configurazione del modello su training e validation set.

<u>Nota:</u> prima di eseguire la cella sottostante assicurarsi di aver eseguito la cella precedente altrimenti le variabili *epoch_count* e *batch_size* non saranno dichiarate.


In [ ]:
history=model.fit(train_x,train_y,validation_data=(validation_x,validation_y),batch_size=batch_size,epochs=epoch_count,verbose=1)

Il metodo **fit(...)** restituisce un oggetto (*history*) che contiene per ogni epoca i valori della *loss function*, dell'accuratezza e dell'indice di *Jaccard* (impostate nel metodo **compile(...)**) sia per il training sia per il validation set.

Eseguire la cella seguente per graficare tali valori.

In [ ]:
indicator_list=['loss','acc','jaccard_index']

_, axs = plt.subplots(1,len(indicator_list),figsize=(8*len(indicator_list),5))

for idx,indicator in enumerate(indicator_list):
  minVal=min(np.amin(history.history[indicator]),np.amin(history.history['val_'+indicator]))
  maxVal=min(np.amax(history.history[indicator]),np.amax(history.history['val_'+indicator]))
  axs[idx].plot(range(1,epoch_count+1),history.history[indicator],'r--',label='Train')
  axs[idx].plot(range(1,epoch_count+1),history.history['val_'+indicator],'b-',label='Val')
  axs[idx].set_xlabel('Epoch')
  axs[idx].set_ylabel(indicator)
  axs[idx].set_ylim([minVal*0.8,maxVal*1.2])
  axs[idx].legend(loc='upper right')

## **Salvataggio del modello migliore**
Per salvare il modello è possibile utilizzare il metodo [**save(...)**](https://keras.io/getting-started/faq/#how-can-i-save-a-keras-model) dandogli in input il percorso in cui salvare il modello. Eseguendo la cella sottostante il modello verrà salvato in remoto su Colab.

In [ ]:
model.save('/content/UNetW.h5')

Utilizzare il metodo **keras.models.load_model(...)** per poter caricare un modello precedentemente salvato.

<u>Nota:</u> se il modello salvato conteneva delle metriche aggiuntive (es. indice di *Jaccard*) in fase di caricamento si dovranno passare queste metriche in maniera esplicita.

In [ ]:
dependencies = {'jaccard_index': jaccard_index}
loaded_model=keras.models.load_model('/content/UNetW.h5',custom_objects=dependencies)

# **Valutazione delle prestazioni sul validation set**
Per eseguire la segmentazione utilizzare il metodo **predict(...)** passandogli in input le immagini da analizzare. Il metodo restituirà un array di segmentation map, una per ogni immagine di input.

In [ ]:
validation_y_preds=model.predict(validation_x)
print('Shape prediction map: ',validation_y_preds.shape)
print('Tipo prediction map: ',validation_y_preds.dtype)

## **Selezione della soglia di binarizzazione ottimale**
Ogni elemento di una prediction map riporta la probabilità che il corrispondente pixel dell'immagine di input appartenga al foreground.

Prima di poter confrontare le prediction map con il ground truth sarà necessario binarizzarle. Per farlo la soluzione più semplice è quella di utilizzare una soglia per decidere la classe di appartenenza di ogni pixel. Ovviamente, le prestazioni della rete varieranno al variare della soglia di binarizzazione utilizzata per cui la scelta della soglia ottima è importante al fine di valutare le prestazioni finali del sistema.

Eseguendo la cella sottostante sarà calcolata l'accuratezza e l'indice di *Jaccard* sul validation set al variare della soglia di binarizzazione utilizzata.

In [ ]:
print('Thr\tAcc\tJac')

# Range delle possibili soglie di binarizzazione
for bin_thr in np.arange(0.0, 1.0, 0.1):
  # Binarizzazione delle prediction map utilizzando la soglia corrente
  thr_bin_y_preds=(validation_y_preds>=bin_thr).astype(np.int64)
  thr_acc_scores=[]
  thr_jac_scores=[]
  for i in range(validation_y_preds.shape[0]):
    acc_score=sklearn.metrics.accuracy_score(validation_y[i].ravel(),thr_bin_y_preds[i].ravel())
    if (np.sum(validation_y[i])!=0 or np.sum(thr_bin_y_preds[i])!=0):
      jac_score=sklearn.metrics.jaccard_score(validation_y[i].ravel(),thr_bin_y_preds[i].ravel())
    else:
      jac_score=1
    thr_acc_scores.append(acc_score)
    thr_jac_scores.append(jac_score)

  print('%.2f\t%.4f\t%.4f' % (bin_thr,sum(thr_acc_scores)/len(thr_acc_scores),sum(thr_jac_scores)/len(thr_jac_scores)))

## **Visualizzazione dei migliori risultati**
Eseguendo la cella sottostante sarà possibile visualizzare le migliori prediction map ottenute dal modello.

Prima di eseguire la cella ricordarsi di impostare i seguenti parametri:
- la soglia di binarizzazione ottimale trovata al passo precedente (*validation_bin_thr*);
- la metrica da utilizzare per selezionare le migliori prediction map (*validation_metric* - 'Acc' per accuratezza e 'Jac' per indice di *Jaccard*);
- il numero di immagini che si vuole visualizzare (*validation_image_to_show*).

In [ ]:
validation_bin_thr=0.5       # Soglia di binarizzazione ottima individuata al passo precedente
validation_metric='Acc'      # Metrica utilizzata per ordinare i risultati 'Acc' o 'Jac'
validation_image_to_show=10  # Numero di risultati da visualizzare

validation_bin_y_preds=(validation_y_preds>=validation_bin_thr).astype(np.int64)

validation_scores=[]
for i in range(validation_y_preds.shape[0]):
    if (validation_metric=='Acc'):
      score=sklearn.metrics.accuracy_score(validation_y[i].ravel(),validation_bin_y_preds[i].ravel())
    elif (validation_metric=='Jac'):
      if (np.sum(validation_y[i])!=0 or np.sum(validation_bin_y_preds[i])!=0):
        score=sklearn.metrics.jaccard_score(validation_y[i].ravel(),validation_bin_y_preds[i].ravel())
      else:
        score=1.0
    validation_scores.append((i,score))

validation_scores.sort(key=lambda elem: elem[1],reverse=True)

_, axs = plt.subplots(validation_image_to_show, 4,figsize=(10, 3*validation_image_to_show))
for i in range(validation_image_to_show):
  image_idx=validation_scores[i][0]
  axs[i,0].imshow(validation_x[image_idx]),axs[i,0].axis('off'),axs[i,0].set_title('%d: %.3f'%(image_idx,validation_scores[i][1]))
  axs[i,1].imshow(np.squeeze(validation_y[image_idx],axis=2),cmap='gray'),axs[i,1].axis('off'),axs[i,1].set_title('Mask')
  axs[i,2].imshow(np.squeeze(validation_bin_y_preds[image_idx],axis=2),cmap='gray',vmin=0,vmax=1),axs[i,2].axis('off'),axs[i,2].set_title('Bin prediction map')
  axs[i,3].imshow(np.squeeze(validation_y_preds[image_idx],axis=2),cmap='gray',vmin=0,vmax=1),axs[i,3].axis('off'),axs[i,3].set_title('Prediction map')

## **Visualizzazione dei peggiori risultati**
In maniera analoga, eseguendo la cella sottostante sarà possibile visualizzare le peggiori prediction map ottenute dal modello.

<u>Nota:</u> Saranno utilizzati gli stessi parametri impostati nella cella precedente.

In [ ]:
validation_scores.sort(key=lambda elem: elem[1])

_, axs = plt.subplots(validation_image_to_show, 4,figsize=(10, 3*validation_image_to_show))
for i in range(validation_image_to_show):
  image_idx=validation_scores[i][0]
  axs[i,0].imshow(validation_x[image_idx]),axs[i,0].axis('off'),axs[i,0].set_title('%d: %.3f'%(image_idx,validation_scores[i][1]))
  axs[i,1].imshow(np.squeeze(validation_y[image_idx],axis=2),cmap='gray'),axs[i,1].axis('off'),axs[i,1].set_title('Mask')
  axs[i,2].imshow(np.squeeze(validation_bin_y_preds[image_idx],axis=2),cmap='gray',vmin=0,vmax=1),axs[i,2].axis('off'),axs[i,2].set_title('Bin prediction map')
  axs[i,3].imshow(np.squeeze(validation_y_preds[image_idx],axis=2),cmap='gray',vmin=0,vmax=1),axs[i,3].axis('off'),axs[i,3].set_title('Prediction map')

# **Ottimizzazione degli iperparametri**
Ripetere la fase di addestramento e valutazione più volte per individuare un insieme di iperparametri ottimali. Si consiglia di intervenire sui seguenti iperparametri (in ordine di priorità):
1. il numero di filtri da utilizzare in ogni blocco di *downsampling* e di *upsampling* (*block_filter_count*);
2. il numero di epoche di addestramento (*epoch_count*);
3. l'ottimizzatore (*optimizer*) utilizzato dalla rete per implementare la *back propagation*. [Keras](https://keras.io/api/optimizers/#available-optimizers) mette a disposizione gli ottimizzatori più comuni;
4. i singoli parametri dell'ottimizzatore (es. *learning rate*);
5. il numero di pattern all'interno di ogni *minibatch* (*batch_size*);
6. la soglia di binarizzazione delle segmentation map (*optimal_bin_thr*).

# **Valutazione delle prestazioni sul test set**
Misurare le prestazioni sul dataset di test della miglior soluzione ottenuta per verificarne l'effettiva capacità di generalizzazione.

## **Test set**
Eseguendo la cella sottostante il dataset di test sarà caricato in memoria.

In [ ]:
db_path='/content/Test'
image_ext='.bmp'

# Caricamento delle immagini
print('Caricamento in corso ...')
test_x,test_y=vr_utilities.load_image_dataset_with_masks(db_path+'/Images',db_path+'/Masks','*'+image_ext)

# Normalizzazione
test_x=(test_x/255)
test_y=(test_y/255)
test_x=test_x.astype(np.float32)
test_y=test_y.astype(np.float32)

print('Shape immagini di test: ',test_x.shape)
print('Shape maschere di test: ',test_y.shape)

Eseguendo la cella sottostante verrà visualizzata un'immagine del test set scelta casualmente (con la rispettiva maschera).

In [ ]:
# Visualizzazione di un'immagine casuale
random_idx=random.randint(0,test_x.shape[0])
_, axs = plt.subplots(1, 2,figsize=(10, 5))
axs[0].imshow(test_x[random_idx]),axs[0].axis('off'),axs[0].set_title('Image')
axs[1].imshow(np.squeeze(test_y[random_idx],axis=2),cmap='gray'),axs[1].axis('off'),axs[1].set_title('Mask')
plt.show()

## **Stima delle prediction map**
Eseguendo la cella sottostante si segmenteranno le immagini del test set utilizzando il modello attuale (*model*).

<u>Nota:</u> prima di eseguire la cella assicurarsi che la variabile *model* contenga la rete ottenuta utilizzando gli iperparametri ottimi.

In [ ]:
test_y_preds=model.predict(test_x)
print('Shape prediction map: ',test_y_preds.shape)
print('Tipo prediction map: ',test_y_preds.dtype)

## **Calcolo delle prestazioni**
Eseguendo la cella sottostante sarà calcolata l'accuratezza e l'indice di *Jaccard* sul test set utilizzando la soglia di binarizzazione ottimale individuata in fase di addestramento.

<u>Nota:</u> prima di eseguire la cella assicurarsi di avere impostato correttamente la soglia di binarizzazione ottimale (*optimal_bin_thr*).

In [ ]:
# Soglia di binarizzazione ottimale
optimal_bin_thr=0.5

# Binarizzazione delle prediction map utilizzando la soglia ottimale
test_bin_y_preds=(test_y_preds>=optimal_bin_thr).astype(np.int64)

test_acc_scores=[]
test_jac_scores=[]
for i in range(test_y_preds.shape[0]):
  acc_score=sklearn.metrics.accuracy_score(test_y[i].ravel(),test_bin_y_preds[i].ravel())
  if (np.sum(test_y[i])!=0 or np.sum(test_bin_y_preds[i])!=0):
    jac_score=sklearn.metrics.jaccard_score(test_y[i].ravel(),test_bin_y_preds[i].ravel())
  else:
    jac_score=1
  test_acc_scores.append(acc_score)
  test_jac_scores.append(jac_score)

print('Acc: %.4f Jac: %.4f'%(sum(test_acc_scores)/len(test_acc_scores),sum(test_jac_scores)/len(test_jac_scores)))

## **Visualizza dei migliori risultati**
Eseguendo la cella sottostante sarà possibile visualizzare le migliori prediction map ottenute dal modello.

Prima di eseguire la cella ricordarsi di impostare i seguenti parametri:
- la metrica da utilizzare per selezionare le migliori prediction map (*test_metric* - 'Acc' per accuratezza e 'Jac' per indice di Jaccard);
- il numero di immagini che si vuole visualizzare (*test_image_to_show*).

In [ ]:
test_metric='Acc'     # Metrica utilizzata per ordinare i risultati 'Acc' o 'Jac'
test_image_to_show=10 # Numero di risultati da visualizzare

test_scores=[]
for i in range(test_y_preds.shape[0]):
    if (test_metric=='Acc'):
      score=sklearn.metrics.accuracy_score(test_y[i].ravel(),test_bin_y_preds[i].ravel())
    elif (test_metric=='Jac'):
      if (np.sum(test_y[i])!=0 or np.sum(test_bin_y_preds[i])!=0):
        score=sklearn.metrics.jaccard_score(test_y[i].ravel(),test_bin_y_preds[i].ravel())
      else:
        score=1
    test_scores.append((i,score))

test_scores.sort(key=lambda elem: elem[1],reverse=True)

_, axs = plt.subplots(test_image_to_show, 4,figsize=(10, 3*test_image_to_show))
for i in range(test_image_to_show):
  image_idx=test_scores[i][0]
  axs[i,0].imshow(test_x[image_idx]),axs[i,0].axis('off'),axs[i,0].set_title('%d: %.3f'%(image_idx,test_scores[i][1]))
  axs[i,1].imshow(np.squeeze(test_y[image_idx],axis=2),cmap='gray'),axs[i,1].axis('off'),axs[i,1].set_title('Mask')
  axs[i,2].imshow(np.squeeze(test_bin_y_preds[image_idx],axis=2),cmap='gray',vmin=0,vmax=1),axs[i,2].axis('off'),axs[i,2].set_title('Bin prediction map')
  axs[i,3].imshow(np.squeeze(test_y_preds[image_idx],axis=2),cmap='gray',vmin=0,vmax=1),axs[i,3].axis('off'),axs[i,3].set_title('Prediction map')

## **Visualizzazione dei peggiori risultati**
In maniera analoga, eseguendo la cella sottostante sarà possibile visualizzare le peggiori prediction map ottenute dal modello.

<u>Nota:</u> Saranno utilizzati gli stessi parametri impostati nella cella precedente.

In [ ]:
test_scores.sort(key=lambda elem: elem[1])

_, axs = plt.subplots(test_image_to_show, 4,figsize=(10, 3*test_image_to_show))
for i in range(test_image_to_show):
  image_idx=test_scores[i][0]
  axs[i,0].imshow(test_x[image_idx]),axs[i,0].axis('off'),axs[i,0].set_title('%d: %.3f'%(image_idx,test_scores[i][1]))
  axs[i,1].imshow(np.squeeze(test_y[image_idx],axis=2),cmap='gray'),axs[i,1].axis('off'),axs[i,1].set_title('Mask')
  axs[i,2].imshow(np.squeeze(test_bin_y_preds[image_idx],axis=2),cmap='gray',vmin=0,vmax=1),axs[i,2].axis('off'),axs[i,2].set_title('Bin prediction map')
  axs[i,3].imshow(np.squeeze(test_y_preds[image_idx],axis=2),cmap='gray',vmin=0,vmax=1),axs[i,3].axis('off'),axs[i,3].set_title('Prediction map')